# Project: Stochastic Gradient Hamiltonian Monte Carlo

In this project we are going to implement standard HMC, HMC with MH, Naive SGHMC and SGHMC with Friction.

# standard HMC

### Basic settings

Suppose we want to sample from the posterior distribution: 
$$p(\theta|D) \propto exp(-U(\theta))$$ where $D = \{x_{1\:n}\}$
which are independent and $U$ is the potential energy fucntion: 
$$U = -\sum_{x_i \in D}\log p(x|\theta)- \log p(\theta)$$
We will sample from joint distribution $$\pi(\theta, r) \propto exp(-U(\theta)-\frac{1}{2}r^TM^{-1}r)$$
where r is the auxiliary momentum variables and M is the mass matrix. They define the kinetic energy together.
Then we discard r and keep $\theta$.

The Hamiltonian function is:
$$H(\theta, r)=U(\theta)+\frac{1}{2}r^TM^{-1}r$$
The Hamiltonian dynamics are:
$$d\theta = M^{-1}r dt\\ dr=-\triangledown U(\theta) dt$$

### Implement HMC

We use a univariate $\theta$ for illustration. Suppose $U(\theta)=-2\theta^2+\theta^4$. Based on $U(\theta)$, we need to define functions $\triangledown U, H$.

In [64]:
import numpy as np
import matplotlib.pyplot as plt

In [53]:
# need to be set manually
def U(theta):
    '''compute U'''
    
    return -2*theta**2 + theta**4

In [54]:
# need to be set manually
def grad_U(theta):
    '''compute gradient of U'''
    
    return -4*theta+4*theta**3

In [55]:
# need to be set manually
def set_M(theta):
    '''initialize M as identity with resonable dimensions based on theta, which is a np.array'''
    
    return np.eye(theta.shape[-1])

In [56]:
# need to be set manually
def H(theta, r, M):
    '''compute Hamiltonian function'''
    assert M.shape[0]==M.shape[1], 'M is not a square matrix'
    
    return U(theta) + 1/2* r.T @ np.linalg.inv(M) @ r

In [57]:
def std_HMC(theta0, epsilon, nmc, max_iteration):
    '''
    implement standard HMC
    theta0: np.array
    epsilon: float
    '''
    
    theta_post = [theta0]
    M = set_M(theta0)
    m = M.shape[0] # number of parameters
    
    i = 1
    while i < (nmc+1):
        r = ri = np.random.multivariate_normal(np.zeros(m), M).reshape(-1,1)
        
        theta= theta0
        
        #simulate discretization of Hamiltonian Dynamics
        r = r - epsilon * grad_U(theta)/2
        
        for j in range(max_iteration):
            theta = theta + epsilon * np.linalg.inv(M) @ r
            r = r - epsilon * grad_U(theta)
        
        # MH correction
        u = np.random.rand()
        ro = np.exp(H(theta, r, M) - H(theta_post[i-1], ri, M))
        if u < min(1,ro):
            theta_post.append(theta)
            i += 1
            
    return theta_post[1:]

In [62]:
x = std_HMC(np.array([1]), 0.01, 1000, 100)

[2, 3]

4